# Creating first agent

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import MessageRole

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   - curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   - az login



In [ ]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Create the agent

In [ ]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )

print(f"Created agent, ID: {agent.id}")

## Create thread

In [ ]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [ ]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [ ]:
user_input = "What are the 5 things I can see in New York?"
run_agent(user_input)

## Cleanup resources

In [ ]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

## Creating an agent and thread

In [ ]:
agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="my-agent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        
    )
print(f"Created agent, ID: {agent.id}")
agent_id = agent.id


## Retrieving an agent

In [ ]:
agent = project_client.agents.get_agent(agent_id)

## Create thread

To retrieve a thread
- thread = project.agents.threads.get("")

In [ ]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

## Run agent

In [ ]:
user_input = "What are the 5 things I can see in San Francisco?"
run_agent(user_input)

## Cleanup resources

In [ ]:
# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")